# Clasificacion no supervisada GEE 

In [1]:
# Importar paquete GEE
import ee
ee.Initialize()

## Filtro zona estudio mediante GEE

In [2]:
# Agregar un shp GEE
zona_ee = ee.FeatureCollection("users/bravomoralesnino/SHP/ZONA_19S")

In [3]:
# Importar geemap
import geemap
Map = geemap.Map(basemap='ESRI')

In [4]:
geometria = zona_ee.geometry()
Map.centerObject(geometria, 11)

In [5]:
Map.addLayer(zona_ee, {"color" : "00FF11"},name = "Zona estudio")
Map

Map(center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['position', 'transparent…

## Selección de imagen

In [6]:
# Seleccionar una imagen Landsat8
img_L8 = ee.Image("LANDSAT/LC08/C01/T1_SR/LC08_003069_20190808")\
             .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'],
                     ['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2'])\
             .multiply(0.0001)\
             .clip(zona_ee)\
             .reproject(crs="EPSG:32719", scale = 30)

In [7]:
# Nombre de las bandas
img_L8.bandNames().getInfo()

['BLUE', 'GREEN', 'RED', 'NIR', 'SWIR1', 'SWIR2']

In [8]:
# Simbologia escala 0 - 1
viz_es = {
    'bands': ['SWIR1','NIR','RED'],
    'min': 0.1,
    'max': 0.6,
    'gamma': 1.6
}

In [9]:
Map.addLayer(img_L8, viz_es,name = "Landsat8 RS")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Entrenamiento ROI

In [10]:
# Crear el conjunto de datos de entrenamiento
Entrenamiento = img_L8.sample(region= geometria,
                               scale = 30,
                               numPixels = 1000) # puntos aleatorios mientras mas mejor

## Aplicar clasificación no supervisada

In [11]:
# Aplicar algoritmo k-mean y entrenarlo
kmeans = ee.Clusterer.wekaKMeans(6).train(Entrenamiento) # 6 clases

In [12]:
# Aplicar clasificacion
class_kmeans = img_L8.cluster(kmeans)

In [13]:
class_kmeans.bandNames().getInfo()

['cluster']

In [14]:
# Simbologia estala 0 - 1
viz_class = {
    'bands': "cluster",
    'min':0,
    'max': 5,
    'palette': ["#ce731e", "#4def4d",
                "#1731b5", "#1c6406",
                "#dced1d", "#de0028"]
}

In [15]:
Map.addLayer(class_kmeans, viz_class,name = "L8 class_kmeans")
Map

Map(bottom=281405.0, center=[-12.908744902436744, -69.9947155021359], controls=(WidgetControl(options=['positi…

## Exportar Imagen

In [16]:
import os
os.getcwd()

'C:\\Users\\GEOMATICA\\Teledeteccion_Python\\GEE'

In [17]:
ruta_archivos = r"E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE"
os.chdir(ruta_archivos) # Cambiar ruta trabajo
os.getcwd() # Consultar ruta trabajo

'E:\\Teledeteccion_GEE\\Python\\Datos_Espaciales\\Resultado_GEE'

In [18]:
# Exportar imagen directorio
geemap.ee_export_image(class_kmeans, filename="L8_class_kmeans.tif", scale=30, region=geometria, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to E:\Teledeteccion_GEE\Python\Datos_Espaciales\Resultado_GEE\L8_class_kmeans.tif


In [19]:
# Exportar en google drive
geemap.ee_export_image_to_drive(class_kmeans, description='L8_class_kmeans', folder='GEE_Python', region=geometria, scale=30)

Exporting L8_class_kmeans ...
